<a href="https://colab.research.google.com/github/ngabo-dev/Formative-2_Data-Preprocessing_Group-6/blob/main/multimodal_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Model 1: Facial Recognition Model (Random Forest)

#### Description:
This model classifies whether a face belongs to a known user or not based on the extracted image features (e.g., histogram, embeddings).


In [62]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
import os

# List files in your main Drive folder
print(os.listdir('/content/drive/MyDrive/'))



['IRAKOZE Jean Paul    Cohort 1 PART 2 English Proficiency Assessment_January_2023.gdoc', 'IRAKOZE Jean Paul Cohort 1  English Proficiency Assessment_January_2023.gdoc', 'Book report (1).gslides', 'Book report.gslides', 'Essay.gdoc', 'IRAKOZE Jean Paul_cohort 6_negpod 5_Personal Reflection Essay..gdoc', 'Copy of (MAKE A COPY)Articulating Process | Journal -01.gdoc', 'Group discussion_Cohort 6_negport5.gdoc', 'IRAKOZE_Jean Paul_Y1T1 Mission Development [RT_Summative]..gdoc', ' Personal SWOT Workbook_Template [Make a copy].gslides', 'Copy of [Make a copy] Personalized Action Plan (1).gdoc', 'Copy of [Make a copy] Personalized Action Plan.gdoc', 'Copy of Copy of 2020_SP_ALU Resume Template [MAKE A COPY]_v3.gdoc', 'IRAKOZE_Jean Paul_negpod5_Personalized Action Plan.gdoc', 'IRAKOZE Jean Paul_negpod5_Personalized Action Plan.gdoc', 'IRAKOZE Jean Paul.gdoc', 'Irakoze jean paul_Martin Luther King Jr. Recording Sheet.gdoc', 'Untitled document (17).gdoc', 'IRAKOZE Jean Paul- Scavenger Hunt Activ

In [64]:
import os

# Set the folder path (adjust if your files are in a subfolder)
folder_path = '/content/drive/MyDrive/'

# List and print each file vertically
for file in os.listdir(folder_path):
    print(file)



IRAKOZE Jean Paul    Cohort 1 PART 2 English Proficiency Assessment_January_2023.gdoc
IRAKOZE Jean Paul Cohort 1  English Proficiency Assessment_January_2023.gdoc
Book report (1).gslides
Book report.gslides
Essay.gdoc
IRAKOZE Jean Paul_cohort 6_negpod 5_Personal Reflection Essay..gdoc
Copy of (MAKE A COPY)Articulating Process | Journal -01.gdoc
Group discussion_Cohort 6_negport5.gdoc
IRAKOZE_Jean Paul_Y1T1 Mission Development [RT_Summative]..gdoc
 Personal SWOT Workbook_Template [Make a copy].gslides
Copy of [Make a copy] Personalized Action Plan (1).gdoc
Copy of [Make a copy] Personalized Action Plan.gdoc
Copy of Copy of 2020_SP_ALU Resume Template [MAKE A COPY]_v3.gdoc
IRAKOZE_Jean Paul_negpod5_Personalized Action Plan.gdoc
IRAKOZE Jean Paul_negpod5_Personalized Action Plan.gdoc
IRAKOZE Jean Paul.gdoc
Irakoze jean paul_Martin Luther King Jr. Recording Sheet.gdoc
Untitled document (17).gdoc
IRAKOZE Jean Paul- Scavenger Hunt Activity.gdoc
Copy of 2020_SP_ALU Resume Template [MAKE A COP

In [72]:
from google.colab import files
uploaded = files.upload()


Saving audio_features.csv to audio_features.csv
Saving customer_social_profiles.csv to customer_social_profiles.csv
Saving customer_transactions.csv to customer_transactions.csv
Saving image_features (1).csv to image_features (1) (3).csv
Saving merged_engineered_data.csv to merged_engineered_data.csv


In [73]:
print(image_df.columns.tolist())


['member_name', 'expression', 'augmentation_type', 'image_path', 'height', 'width', 'aspect_ratio', 'total_pixels', 'blue_mean', 'blue_std', 'blue_min', 'blue_max', 'blue_median', 'green_mean', 'green_std', 'green_min', 'green_max', 'green_median', 'red_mean', 'red_std', 'red_min', 'red_max', 'red_median', 'hue_mean', 'hue_std', 'saturation_mean', 'saturation_std', 'value_mean', 'value_std', 'gray_mean', 'gray_std', 'gray_min', 'gray_max', 'gray_median', 'blue_hist_0', 'blue_hist_1', 'blue_hist_2', 'blue_hist_3', 'blue_hist_4', 'blue_hist_5', 'blue_hist_6', 'blue_hist_7', 'blue_hist_8', 'blue_hist_9', 'blue_hist_10', 'blue_hist_11', 'blue_hist_12', 'blue_hist_13', 'blue_hist_14', 'blue_hist_15', 'green_hist_0', 'green_hist_1', 'green_hist_2', 'green_hist_3', 'green_hist_4', 'green_hist_5', 'green_hist_6', 'green_hist_7', 'green_hist_8', 'green_hist_9', 'green_hist_10', 'green_hist_11', 'green_hist_12', 'green_hist_13', 'green_hist_14', 'green_hist_15', 'red_hist_0', 'red_hist_1', 'red_

!

## 1. Facial Recognition Model (Random Forest)

In [90]:
%%writefile models/facial_model.py
# models/facial_model.py

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import joblib
import os

def train_facial_model():
    df = pd.read_csv("image_features.csv")

    label_col = 'member_name'  # use actual column name from your dataset

    # Validate label column presence
    if label_col not in df.columns:
        raise KeyError(f"The '{label_col}' column is missing in the image_features.csv")

    # Drop rows with missing label values
    df = df.dropna(subset=[label_col])

    # Encode label column (target)
    label_encoder = LabelEncoder()
    df[label_col] = label_encoder.fit_transform(df[label_col])

    # Separate features and labels
    X = df.drop(columns=[label_col])
    y = df[label_col]

    # Encode any non-numeric feature columns
    for col in X.columns:
        if X[col].dtype == 'object':
            X[col] = LabelEncoder().fit_transform(X[col])

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Save model and label encoder
    os.makedirs("saved_models", exist_ok=True)
    joblib.dump(model, "saved_models/facial_model.pkl")
    joblib.dump(label_encoder, "saved_models/facial_label_encoder.pkl")

    print("✅ Facial recognition model trained and saved successfully!")

if __name__ == "__main__":
    train_facial_model()


Overwriting models/facial_model.py


In [91]:
!python3 models/facial_model.py

✅ Facial recognition model trained and saved successfully!


## 2. Audio Recognition Model (Random Forest)

In [95]:
%%writefile models/voice_model.py
# Voiceprint Verification Model - Random Forest

import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import joblib

def train_voice_model():
    df = pd.read_csv("/content/drive/MyDrive/audio_features.csv")

    # ✅ Extract label from filename (e.g. 'john_01.wav' -> 'john')
    if 'filename' not in df.columns:
        raise KeyError("The 'filename' column is missing in audio_features.csv")

    df['label'] = df['filename'].apply(lambda x: os.path.basename(x).split('_')[0])

    # ✅ Encode label
    le = LabelEncoder()
    df['label'] = le.fit_transform(df['label'])

    # ✅ Features (drop filename and label)
    X = df.drop(columns=['filename', 'label'])

    # ✅ Handle non-numeric types in features
    for col in X.columns:
        if X[col].dtype == 'object':
            X[col] = LabelEncoder().fit_transform(X[col])

    y = df['label']

    # ✅ Split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # ✅ Train
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # ✅ Report
    y_pred = model.predict(X_test)
    print("🎤 Voice Recognition Model Report:")
    print(classification_report(y_test, y_pred, target_names=le.classes_))

    # ✅ Save
    os.makedirs("saved_models", exist_ok=True)
    joblib.dump(model, "saved_models/voice_model.pkl")
    joblib.dump(le, "saved_models/voice_label_encoder.pkl")

    print("✅ Voice model trained and saved successfully.")

if __name__ == "__main__":
    train_voice_model()


Overwriting models/voice_model.py


In [96]:
!python3 models/voice_model.py

🎤 Voice Recognition Model Report:
              precision    recall  f1-score   support

        omar       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

✅ Voice model trained and saved successfully.


In [94]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/audio_features.csv")
print(df.columns.tolist())


['mfccs', 'rolloff', 'energy', 'filename']


## 3. Product Recognition Model (Random Forest)

In [98]:
%%writefile models/product_model.py

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
import joblib
import os

def train_product_model():
    # Load data
    df = pd.read_csv("merged_engineered_data.csv")  # Adjust path if needed

    label_col = "product_category"
    if label_col not in df.columns:
        raise KeyError(f"'{label_col}' column not found in dataset")

    df = df.dropna(subset=[label_col])  # Remove missing labels

    # Encode all object columns
    label_encoders = {}
    for col in df.columns:
        if df[col].dtype == 'object':
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col].astype(str))
            label_encoders[col] = le

    # Features & labels
    X = df.drop(label_col, axis=1)
    y = df[label_col]

    # ✅ Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Split
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42
    )

    # Train
    model = LogisticRegression(max_iter=5000)  # Increased max_iter
    model.fit(X_train, y_train)

    # Evaluate
    y_pred = model.predict(X_test)
    print("🛍️ Product Recommendation Model Report:")
    print(classification_report(y_test, y_pred))

    # ✅ Save model, scaler, encoders
    os.makedirs("saved_models", exist_ok=True)
    joblib.dump(model, "saved_models/product_model.pkl")
    joblib.dump(scaler, "saved_models/product_scaler.pkl")
    joblib.dump(label_encoders, "saved_models/product_label_encoders.pkl")

    print("✅ Product model trained and saved successfully.")

if __name__ == "__main__":
    train_product_model()


Overwriting models/product_model.py


In [99]:
!python3 models/product_model.py

🛍️ Product Recommendation Model Report:
              precision    recall  f1-score   support

           0       1.00      0.10      0.18        10
           1       0.29      0.40      0.33         5
           2       0.35      0.60      0.44        10
           3       0.50      0.43      0.46         7
           4       0.33      0.36      0.35        11

    accuracy                           0.37        43
   macro avg       0.49      0.38      0.35        43
weighted avg       0.51      0.37      0.35        43

✅ Product model trained and saved successfully.


In [47]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/merged_engineered_data.csv")
print(df.columns.tolist())


['customer_id_new', 'social_media_platform', 'engagement_score', 'purchase_interest_score', 'review_sentiment', 'customer_id_new_numeric', 'customer_id_legacy', 'transaction_id', 'purchase_amount', 'purchase_date', 'product_category', 'customer_rating', 'purchase_month', 'purchase_day_of_week', 'total_purchase_amount', 'number_of_transactions', 'average_customer_rating']


In [82]:
import pandas as pd

df = pd.read_csv("image_features.csv")
print(df.columns.tolist())


['member_name', 'expression', 'augmentation_type', 'image_path', 'height', 'width', 'aspect_ratio', 'total_pixels', 'blue_mean', 'blue_std', 'blue_min', 'blue_max', 'blue_median', 'green_mean', 'green_std', 'green_min', 'green_max', 'green_median', 'red_mean', 'red_std', 'red_min', 'red_max', 'red_median', 'hue_mean', 'hue_std', 'saturation_mean', 'saturation_std', 'value_mean', 'value_std', 'gray_mean', 'gray_std', 'gray_min', 'gray_max', 'gray_median', 'blue_hist_0', 'blue_hist_1', 'blue_hist_2', 'blue_hist_3', 'blue_hist_4', 'blue_hist_5', 'blue_hist_6', 'blue_hist_7', 'blue_hist_8', 'blue_hist_9', 'blue_hist_10', 'blue_hist_11', 'blue_hist_12', 'blue_hist_13', 'blue_hist_14', 'blue_hist_15', 'green_hist_0', 'green_hist_1', 'green_hist_2', 'green_hist_3', 'green_hist_4', 'green_hist_5', 'green_hist_6', 'green_hist_7', 'green_hist_8', 'green_hist_9', 'green_hist_10', 'green_hist_11', 'green_hist_12', 'green_hist_13', 'green_hist_14', 'green_hist_15', 'red_hist_0', 'red_hist_1', 'red_